# Data Augmentation

In this notebooks we'll develop the functions required to change tagged and untagged strings from a text in order to recreate a bigger dataset. It is imperative to retain the information of the tags, that may change due to differences in lenght.

## Imports
### Libraries

In [48]:
import re, os, json, pickle
import spacy
from spacy import displacy
import pandas as pd
import numpy as np

### Datasets

In [49]:
data_dir = '../../data/estatutos/tagged/spacy_dataset_2020-5-6.pkl'

In [50]:
with open(data_dir, 'rb') as file:
    dataset = pickle.load(file)

## Dataset structure and objective function
As we can see below, we have a dictionary with the name of the document as keys and more dictionaries as values.
The structure is as follows:
```python
{
    "company_name" :{
        "doc_id" : "original_name",
        "pages" : {
            'page_number': {
                'text_body_coords': [
                    [int, int],
                    [int, int],
                    [int, int],
                    [int, int]
                ],
                'lectura': 'text_containing in that page'
            }
        },
        'text' : 'the concatenated text string of every page',
        'entities': {
            'search': ['list', 'of', 'entities', 'available', 'to', 'tag'],
            'tags': [
                {
                    'tag': 'name_of_tag',
                    'start': int, # where the entity starts on the string
                    'finish': int, # same logic as above
                    'text': 'string', # with the slice of the text marked before
                    'value': 'string', # what it would look like after parsing
                    'extra1': 'string', # not used
                    'extra2': 'string', #not used
                    'index': int, # for multi entity tags
                }
            ]
        }
         
    }

}
```

#### We're interested in modifying this part
```python
{
    'doc_id' : 'original_name',
    'pages' : {
        'page_number': {
            'text_body_coords': [
                [int, int],
                [int, int],
                [int, int],
                [int, int]
            ],
            'lectura': 'text_containing in that page'
        }
    },
    'text' : 'the concatenated text string of every page',
    'entities': {
        'search': ['list', 'of', 'entities', 'available', 'to', 'tag'],
        'tags': [
            {
                'tag': 'name_of_tag',
                'start': int, # where the entity starts on the string
                'finish': int, # same logic as above
                'text': 'string', # with the slice of the text marked before
                'value': 'string', # what it would look like after parsing
                'extra1': 'string', # not used
                'extra2': 'string', #not used
                'index': int, # for multi entity tags
            }
        ]
    }
}
```


#### In order to achieve this, we'll create a class that takes as input that dictionary and modifies accordingly.

In [51]:
d1 = dataset['confecom_srl.']

##### We delete the key that will not be currently used

In [5]:
d1.pop('pages')

{'0': {'text_body_coords': [[237, 348],
   [1178, 332],
   [260, 1686],
   [1201, 1671]],
  'lectura': 'O DE SOCIEDAD DE RESPONSABILIDAD LIMITADA. “CONFECOM S.\n\n12)\n\n \n\nROA, argentina, casada en primeras nupcias con Raul Alberto LESCA-\n\nNO, nacida el 10 de marzo de 1966, comerciante, titular del docu-\nmento nacional de identidad número 17.636.488, C.U.I.T.: 27-\n17636488-8, y Yamila Belén LESCANO, argentina, soltera, nacida el\n6 de marzo de 1998, comerciante, titular del documento nácional de\nidentidad número 41.048.026, C.U.I.L.: 27-41048026-9, ambos domici-\nliados en Pico 4785, Ciudad Autónoma de Buenos Aires; resuelven de-\njar constituida una sociedad de responsabilidad limitada que se re-\n\ngirá por la Ley número 19.550/22.903 y el siguiente estatuto:\n\nPRIMERA: La sociedad se denomina: "CONFECOM S.R.L."” y tiene su do-\nmicilio legal en jurisdicción de la Ciudad Autónoma de Buenos Ai-\nres, pudiendo establecer agencias o sucursales en cualquier lugar\n\ndel país o d

### Visualization
We'll create a simple function to transform data to be displayed in notebooks or html using displacy. To achieve this, the final shape of the dictionary should be:

```python
{
    "text": "But Google is starting from behind.",
    "ents": [{"start": 4, "end": 10, "label": "ORG"}],
    "title": None
}
```

In [6]:
d1['entities'].pop('search')

['firmantes_nombre',
 'firmantes_dni',
 'socios_nombre',
 'razon_social',
 'tipicidad',
 'vigencia',
 'capital',
 'num_escritura',
 'fecha_contrato',
 'mandato_dir',
 'tipo_rep_legal',
 'facultades_firm',
 'aporte_socios',
 'fecha_cierre_ejercicio',
 'objeto_social',
 'cargo_firmantes',
 'legal_domicilio',
 'sede_domicilio',
 'firma_domicilio',
 'poderes_especiales',
 'reservas_legales',
 'garantias_directorio',
 'aumentos_capital',
 'firmante_domicilio',
 'firmante_familiar',
 'fecha_nacimiento',
 'fecha_final_doc']

### We check our output

In [14]:
class TaggedDoc:
    def __init__(self, document):
        self.document = document
        self.spacy_ents = self._displacy_transform().get('ents')
        self.title = document.get('doc_id')
        self.ents = document.get('entities').get('tags')
        self.text = document.get('text')
        self.text_len = len(self.text)
        self.ents_df = pd.DataFrame(self.ents)
        
    def _displacy_transform(self) -> dict:
        ent_list = []
        for tag in self.document['entities']['tags']:
            ent_list.append(
                {
                    'start' : tag.get('start'),
                    'end' : tag.get('end'),
                    'label': tag.get('tag')
                }
            )

        return {
            'text' : self.document.get('text'),
            'ents' : ent_list,
            'title' : self.document.get('doc_id') 
        }
    
    def render(self, style='ent', jupyter=True, manual=True, page=False):
        displacy.render(self._displacy_transform(), style=style, jupyter=jupyter, manual=manual, page=page)
        
        
    def index_augmentation(df_new_ents):
        new_ents = df_new_ents.copy()
        new_ents = new_ents.sort_values('start')
        new_ents.index = range(new_ents.shape[0])
        new_ents['len'] = new_ents.text.apply(len)
        new_ents['new_len'] = new_ents.new_text.apply(len)
        new_ents['diff'] = new_ents['new_len'] - new_ents['len']
        indecis = new_ents.loc[new_ents.diff != 0].index.to_list()

        for index in indecis:
            diff = new_ents.loc[index, 'diff']
            new_ents.loc[index, 'end'] = new_ents.loc[index, 'end'] + diff
            new_ents.loc[index+1:,'start'] = new_ents.loc[index+1:,'start'] + diff
            new_ents.loc[index+1:,'end'] = new_ents.loc[index+1:,'end'] + diff

        return [tagged_entity for tagged_entity in new_ents.T.to_dict().values()]
    
    

In [15]:
doc = TaggedDoc(d1)

In [17]:
doc.spacy_ents

[{'start': 312, 'end': 330, 'label': 'fecha_nacimiento'},
 {'start': 138, 'end': 157, 'label': 'fecha_nacimiento'},
 {'start': 106, 'end': 126, 'label': 'firmante_familiar'},
 {'start': 223, 'end': 233, 'label': 'firmantes_dni'},
 {'start': 261, 'end': 281, 'label': 'firmantes_nombre'},
 {'start': 396, 'end': 406, 'label': 'firmantes_dni'},
 {'start': 438, 'end': 496, 'label': 'firmante_domicilio'},
 {'start': 529, 'end': 565, 'label': 'tipicidad'},
 {'start': 669, 'end': 685, 'label': 'razon_social'},
 {'start': 736, 'end': 767, 'label': 'legal_domicilio'},
 {'start': 884, 'end': 886, 'label': 'vigencia'},
 {'start': 973, 'end': 1766, 'label': 'objeto_social'},
 {'start': 1822, 'end': 1845, 'label': 'capital'},
 {'start': 2081, 'end': 2104, 'label': 'tipo_rep_legal'},
 {'start': 2113, 'end': 2145, 'label': 'mandato_dir'},
 {'start': 2146, 'end': 2382, 'label': 'facultades_firm'},
 {'start': 2384, 'end': 2642, 'label': 'garantias_directorio'},
 {'start': 3367, 'end': 3378, 'label': 'fe

--------------------------------------------------------------------
## Augmentation functions
There are 2 main functions to be developed:
1) One or more that create random entities.

2) Another that plugs the new entities into the original text and keeps track of the positions.


We need to create one or more functions that generate new entities.
- Name generator: from AFIP
- ~Date generator: 3 different formats, 366 days = 1098 possibilities.~
- ~Vigencia generator: both spelled and numerical~ 
- Tipicidad generator: 3 types, 6 categories, rather easy.
- ~Fecha cierre ejercicio generator: spelled, numerical and mixed.~
- ~Mandato dir generator: written years, fiscal years, numeric, mixed and different types of undefined.~
- Razon Social generator: from AFIP
- Capital generator: might be tricky
- ~DNI generator: from AFIP~ Random

The rather complicated part is to match strings with different lenghts into the text without loosing track of the position of the others.

### 1) Synthetic Dates
We are going to generate 2 functions:
- One to create dates within a certain time interval.
- The other to change the level of formality

In [59]:
import datetime, random
from tqdm.autonotebook import tqdm, trange

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [60]:
def random_date_generator(start_year:int=1900, end_year:int=2050, start_month:int=1, end_month:int=12, start_day:int=1, end_day:int=31, mapper:dict=None, seed:int=None) -> datetime.date:
    if mapper:
        start_year = mapper['start_year']
        end_year = mapper['end_year']
        start_month = mapper['start_month']
        end_month = mapper['end_month']
        start_day = mapper['start_day']
        end_day = mapper['end_day']
    
    if seed:
        random.seed(seed)
        
    start_date = datetime.date(start_year, start_month, start_day)
    end_date = datetime.date(end_year, end_month, end_day)
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + datetime.timedelta(days=random_number_of_days)
    
    return random_date

In [61]:
from num2words import num2words
def date_formatter(date: datetime.date, formality: str = 'veryformal', include_year: bool = True) -> str:
    
    formality_list = ['basic', 'basic2', 'mixed', 'mixed2', 'regular', 'formal', 'veryformal', 'random']
    
    if formality not in formality_list:
        raise KeyError(f'Keyword `{formality}` not found. Argument `formality` must be one of {formality_list}.')
    elif formality == 'random':
        formality = formality_list[random.randint(0,5)]
    
    month_mapper = {
        '01':'Enero',
        '02':'Febrero',
        '03':'Marzo',
        '04':'Abril',
        '05':'Mayo',
        '06':'Junio',
        '07':'Julio',
        '08':'Agosto',
        '09':'Septiembre',
        '10':'Octubre',
        '11':'Noviembre',
        '12':'Diciembre'}

    day = f'{date.day}' if len(str(date.day)) > 1 else f'0{date.day}'
    month = f'{date.month}' if len(str(date.month)) > 1 else f'0{date.month}'
    
    if formality not in ['basic', 'basic2']:
        day_words = num2words(day, lang='es', to='cardinal') if formality != 'veryformal' else num2words(day, lang='es', to='ordinal')
        month_words = month_mapper[month]
    
    if include_year:
        year = f'{date.year}'    
        year_words = num2words(year, lang='es')
    
    format_mapper = {
        'basic': f'{day}-{month}-{year}' if include_year else f'{day}-{month}',
        'basic2': f'{day}/{month}/{year}' if include_year else f'{day}/{month}',
        'mixed': f'{day} de {month_words} de {year}' if include_year else \
                 f'{day} de {month_words}',
        'mixed2':f'{day} de {month_words} de {year_words}' if include_year else \
                 f'{day} de {month_words}',
        'regular': f'{day_words} de {month_words} de {year_words}' if include_year else \
                   f'{day_words} de {month_words}',
        'formal': f'{day_words} del mes de {month_words} de {year_words}' if include_year\
                  else f'{day_words} del mes de {month_words}',
        'veryformal': f'{day_words} día del mes de {month_words} del año {year_words}' \
                    if include_year else f'{day_words} días del mes de {month_words}'
    }
    result = format_mapper[formality]
    
    return result

### 2) Index augmentation

This function allows us to mantain the relationship between the slices of each entity. For every extra character replaced, the following entities change their start and finish values. The same happens if the new string is shorter.

In [47]:
""" Here you can find a suite of tools to work with tagged documents. You can render them or 
    create synthetic documents to supersample your dataset.

Classes
-------
- TaggedDoc
    Intended to open tagged documents using the Text Tag Tool.
"""

import pandas as pd
from spacy import displacy


class TaggedDoc:
    """
        Takes a tagged document in dictionary format and creates an object.
    
    Attributes
    --------
    - TaggedDoc.document
        Returns the same dictionary that was provided.
    - TaggedDoc.displacy_ents
        Returns a list of dictionaries with every entity tagged. Each dictionary contains keys: 
        `start`, `end` and `label`.
    - TaggedDoc.title
        Returns the `doc_id` of the document provided.
    - TaggedDoc.text
        Returns the entire text of the document provided.
    - TaggedDoc.text_len
        Returns the lenght of the document measured in characters.
    - TaggedDoc.ents_df
        Returns a pandas.DataFrame object with the entities tagged.

    Methods
    -------
    - TaggedDoc.render
    - TaggedDoc.index_augmentation


    
    """
    def __init__(self, document):
        if not isinstance(document, dict):
            raise TypeError(f'This class only takes a dictionary as input. You provided a {type(document)}.')
        elif [key for key in ['doc_id', 'text', 'entities'] if key not in document.keys()]:
            raise KeyError('The dictionary must contain the keys `doc_id`, `text` and `entities`.')
        else:
            self.document = document
            self.displacy_format = self._displacy_transform()
            self.displacy_ents = self.displacy_format.get('ents')
            self.title = document.get('doc_id')
            self.ents = document.get('entities').get('tags')
            self.text = document.get('text')
            self.text_len = len(self.text)
            self.ents_df = pd.DataFrame(self.ents, index=[self.title for x in range(len(self.ents))])


    def _displacy_transform(self) -> dict:
        ent_list = []
        if not isinstance(self.document['entities'], dict):
            raise TypeError(f"The dictionary provided does not contain a dictionary with it's entities tagged.\
                 You provided a {type(self.document['entities'])} instead.")
        
        elif 'tags' not in self.document['entities'].keys():
            raise KeyError(f"You provided a dictionary without the key 'tags'")
        
        else:
            for tag in self.document['entities'].get('tags'):
                ent_list.append(
                    {
                        'start' : tag.get('start'),
                        'end' : tag.get('end'),
                        'label': tag.get('tag')
                    }
                )

            return {
                'text' : self.document.get('text'),
                'ents' : ent_list,
                'title' : self.document.get('doc_id') 
            }
    
    
    def render(self, style='ent', jupyter=True, manual=True, page=False):
        for _ in self.displacy_format.get('ents'):
            if not _.get('start') or not _.get('end') or not _.get('label'):
                to_render = {'text': 'Esto es un texto completo con entidades, pero vos no sabes de donde sacarlo.',
                             'ents': [{'start': 46, 'end': 49, 'label': 'GIL'}],
                             'title': 'Por favor leé la documentación.'}
                break
            else:
                to_render = self.displacy_format
                break
        displacy.render(to_render, style=style, jupyter=jupyter, manual=manual, page=page)        


    def index_augmentation(self):
        new_ents = self.ents_df.copy() # careful with this
        new_ents = new_ents.sort_values('start')
        new_ents.index = range(new_ents.shape[0])
        new_ents['len'] = new_ents.text.apply(len)
        new_ents['new_len'] = new_ents.new_text.apply(len)
        new_ents['diff'] = new_ents['new_len'] - new_ents['len']
        indecis = new_ents.loc[new_ents.diff != 0].index.to_list()

        for index in indecis:
            diff = new_ents.loc[index, 'diff']
            new_ents.loc[index, 'end'] = new_ents.loc[index, 'end'] + diff
            new_ents.loc[index+1:,'start'] = new_ents.loc[index+1:,'start'] + diff
            new_ents.loc[index+1:,'end'] = new_ents.loc[index+1:,'end'] + diff

        return [tagged_entity for tagged_entity in new_ents.T.to_dict().values()]
    
    

### 3) Synthetic Names, DNI's and CUIT's
We're going to create a function that allows us to take a random name from a database taken from AFIP.

In [2]:
import pandas as pd
names = pd.read_csv('../../data/estatutos/external_sources/afip_names_cuits.csv', dtype='str')

In [4]:
import numpy as np
names['name'] = names.concatenado.apply(lambda x: x[:30] if isinstance(x, str) else np.nan)

In [5]:
names['name'] = names['name'].str.replace('\s{2,}', '', regex=True)

In [6]:
names = names.dropna()

#### Companies and people
Since we have companies in the list, we can take them to create synthetic "Razon Social". Keep in mind that we'll include them as people's name since they can be founders of a new company as any regular person can. And we're also going to throw away goverments, municipalities, companies in formation and those who filed bankrupcy.

As we can see, there are 68352 different companies. We'll use that as a mask to create a new dataframe

In [7]:
company_mask = names['name'].str.contains(r'\bSA$|\bSRL$|\bSAS$|\bSAU$|\bSASU$|\bSAIC$|\bSACIF$')

In [8]:
names.loc[company_mask]

,concatenado,cuit,dni,name
68968,DE MIRANDA EURICO SA,20056190415,05619041,DE MIRANDA EURICO SA
138919,DE LAVALLAZ ALBERTO ANTONIO SAACEXA,20078365863,07836586,DE LAVALLAZ ALBERTO ANTONIO SA
269324,GARCIA CASTELLANOS FERNANDO SANINIF,20111927082,11192708,GARCIA CASTELLANOS FERNANDO SA
334800,GRECO FRANCISCO ANTONIO SA NINIA,20120626575,12062657,GRECO FRANCISCO ANTONIO SA
402647,SOLDANO ALVARO GERMAN DEL SA NINIA,20128898558,12889855,SOLDANO ALVARO GERMAN DEL SA
...,...,...,...,...
4955893,BE3 SRL,34686133305,68613330,BE3 SRL
4955894,DIFEMA SRL,34686233318,68623331,DIFEMA SRL
4955895,RUBRO PUBLICIDAD SA,34686233482,68623348,RUBRO PUBLICIDAD SA
4955896,EAGLEMP TURISMO SRL,34686333320,68633332,EAGLEMP TURISMO SRL


We found a problem. The dataset wasn't properly cleaned. As we can see, some people ho have "SA" at the end, might have been part of their names. The good part is that it seems that the data frame is sorted. First people, then companies, then goverment bodies. Let's begin with goverment offices, since we really don't need them.

In [9]:
guv_name = names['concatenado'].str.contains('GOBIER|MUNICIP|UNIVERSIDA|BIBLIO|ASOCIAC|ESCUELA|COLEGIO|HOSPITAL|SANATORIO|CLINICA|INSTITUT|SECRETA|MINISTER|OFICINA')
guv_cuit = names.cuit.str.contains(r'^30|^33|^34')

In [10]:
names.loc[guv_name & guv_cuit]

,concatenado,cuit,dni,name
4482664,ASOCIACION MUTUAL DAN,30500045198,50004519,ASOCIACION MUTUAL DAN
4482667,ASOCIACION ARGENTINA DE COMPA#,30500048782,50004878,ASOCIACION ARGENTINA DE COMPA#
4482685,INSTITUTO AUTARQUICO,30500055509,50005550,INSTITUTO AUTARQUICO
4482696,INSTITUTO COOPERATIVO,30500060588,50006058,INSTITUTO COOPERATIVO
4482700,INSTITUTO ASEGURADOR,30500063242,50006324,INSTITUTO ASEGURADOR
...,...,...,...,...
4955887,BIBLIOTECA POPULAR DE LOS,34684033427,68403342,BIBLIOTECA POPULAR DE LOS
4955891,ASOCIACION CIVIL COMITE DE,34685233411,68523341,ASOCIACION CIVIL COMITE DE
4955899,GOBIERNO DE LA CIUDAD DE,34999032089,99903208,GOBIERNO DE LA CIUDAD DE
4955901,MUNICIPALIDAD DE LA MATANZA,34999257560,99925756,MUNICIPALIDAD DE LA MATANZA


So, those 30653 entities go.

In [11]:
names = names.loc[~(guv_name & guv_cuit)]

We go back to extract **Companies** with our acquired knowledge. Plus, we found that some companies have SAS and others S.A.S., so we'll include those as well

In [12]:
company_mask = names['name'].str.contains(r'\bSA$|\bSRL$|\bSAS$|\bSAU$|\bSASU$|\bSAIC$|\bSACIF$')
company_dots = names['name'].str.contains(r'\bS\.A\.$|\bS\.R\.L\.$|\bS\.A\.S\.$|\bS\.A\.U\.$|\bS\.A\.S\.U\.$|\bS\.A\.I\.C\.$|\bS\.A\.C\.I\.F\.$')
company_spaces = names['name'].str.contains(r'\bS\sA$|\bS\sR\sL$|\bS\sA\sS$|\bS\sA\sU$|\bS\sA\sS\sU$|\bS\sA\sI\sC$|\bS\sA\sC\sI\sF$')
company_cuit = names.cuit.str.contains(r'^30|^33|^34')

In [13]:
companies = names.loc[(company_mask | company_dots | company_spaces) & (company_cuit)]

In [14]:
companies

,concatenado,cuit,dni,name
4482617,BANCO BBVA ARGENTINA S.A.,30500003193,50000319,BANCO BBVA ARGENTINA S.A.
4482625,BANCO DE SAN JUAN S A,30500009442,50000944,BANCO DE SAN JUAN S A
4482627,BANCO DE CORRIENTES SA,30500010602,50001060,BANCO DE CORRIENTES SA
4482634,BANCO DEL CHUBUT S.A.,30500012990,50001299,BANCO DEL CHUBUT S.A.
4482636,CHUBB SEGUROS ARGENTINA S A,30500016260,50001626,CHUBB SEGUROS ARGENTINA S A
...,...,...,...,...
4955893,BE3 SRL,34686133305,68613330,BE3 SRL
4955894,DIFEMA SRL,34686233318,68623331,DIFEMA SRL
4955895,RUBRO PUBLICIDAD SA,34686233482,68623348,RUBRO PUBLICIDAD SA
4955896,EAGLEMP TURISMO SRL,34686333320,68633332,EAGLEMP TURISMO SRL


Now LOOK AT THAT.. We went from 68000 to 205500 companies!

In [15]:
companies = companies.drop('concatenado', axis=1)

In [553]:
companies.to_csv('../../data/estatutos/external_sources/companies.csv', index=False)

In [16]:
persons = names.loc[~((company_mask | company_dots | company_spaces) & (company_cuit)), ['cuit', 'dni', 'name']]

In [23]:
persons = persons.loc[~(persons.name=='') | (persons.name=='  ') | (persons.name=='   ')]

In [25]:
persons.to_csv('../../data/estatutos/external_sources/persons.csv', index=False)

In [29]:
def random_name_generator(name_type:str, include_id=True):
    possible_types = ['company', 'person', 'any']
    if not name_type in possible_types:
        raise KeyError(f'{name_type} is not a valid option. Please choose one of the following {possible_types}')
    
    if name_type == 'company':
        names = pd.read_csv('../../data/estatutos/external_sources/companies.csv', dtype=str).sample(10000)
    elif name_type == 'person':
        names = pd.read_csv('../../data/estatutos/external_sources/persons.csv', dtype=str).sample(10000)
    elif name_type == 'any':
        persons = pd.read_csv('../../data/estatutos/external_sources/persons.csv', dtype=str).sample(10000)
        companies = pd.read_csv('../../data/estatutos/external_sources/companies.csv', dtype=str).sample(10000)
        names = pd.concat([persons, companies])
        del persons, companies
    
    result = names.sample(1).iloc[0]
    if not include_id:
        result = result['name']
    return result

In [34]:
%%timeit
random_name_generator('any')

4.22 s ± 71.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


As we can see, it takes too long to take one single name. We'll come back to this function once we create the dataset.

### 4) Random Mandato
We created a function that returns all the values expected for mandato.
We use `np.random.randint(0,3)` at the begining in order to give 2/3 chances of returning a finite number, and 1/3 chances of returning a permanent goverment of the company.

In [559]:
def mandato_generator():
    if np.random.randint(0,3):
        years = np.random.randint(1,11)
        keywords = ['años', 'ejercicios'][np.random.randint(0,2)]
        years_words = num2words(years, lang='es')
        random_year = [years, years_words, f'{years_words} ({years})', f'{years} ({years_words})'][np.random.randint(0,4)]
        result = f'{random_year} {keywords}'
    else:
        result = ['término de duración de la sociedad', 'plazo de duración de la sociedad', 'vencimiento de la sociedad'][np.random.randint(0,3)]
    
    return result

### 5) Random Vigencia

In [560]:
def vigencia_generator():
    years = np.random.randint(1,101)
    years_words = num2words(years, lang='es')
    salad = [years, years_words, f'{years_words} ({years})', f'{years} ({years_words})'][np.random.randint(0,4)]
    
    return salad

In [365]:
random_vigencia()

'cuarenta y ocho'

### 6) Random tipicidad

In [561]:
def tipicidad_generator():
    company_type = ['sociedad de responsabilidad limitada', 
                    'sociedad anónima',
                    'sociedad por acciones simplificada',
                    'sociedad anónima unipersonal',
                    'sociedad por acciones simplificada unipersonal'][np.random.randint(0, 5)]
    style = ['lower', 'upper', 'title'][np.random.randint(0,3)]
    
    if style == 'lower':
        result = company_type
    elif style == 'upper':
        result = company_type.upper()
    elif style == 'title':
        result = company_type.title()
    
    return result

In [431]:
random_tipicidad()

'SOCIEDAD ANÓNIMA'

### 7) Random DNI and Random CUIT

In [564]:
def id_generator(cuit=False):
    millions = np.random.randint(0,100)
    thousands = np.random.randint(0,1000)
    hundreds = np.random.randint(0,1000)
    
    
    if thousands < 10:
        thousands = f'00{thousands}'
    elif thousands < 100:
        thousands = f'0{thousands}'
    else: 
        thousands = f'{thousands}'

    if hundreds < 10:
        hundreds = f'00{hundreds}'
    elif hundreds < 100:
        hundreds = f'0{hundreds}'
    else: 
        hundreds = f'{hundreds}'
        
    result = f'{millions}.{thousands}.{hundreds}'

    if cuit:
        millions = f'0{millions}' if millions < 10 else f'{millions}'
        beginning = np.random.randint(20,36)
        end = np.random.randint(1,10)
        result = f'{beginning}-{millions}{thousands}{hundreds}-{end}'
    
    return result

### 8) Random Capital